<a href="https://colab.research.google.com/github/SAMI-CODEAI/DeepLearning/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_datasets as tfds

# Load 'cats_vs_dogs' dataset
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
train_dataset = dataset['train']

# Resize and preprocess images
IMG_SIZE = 160
BATCH_SIZE = 32

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)  # MobileNetV2 expects preprocessed input
    return image, label

train_dataset = train_dataset.map(format_example)
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split into train and validation
val_batches = int(0.2 * info.splits['train'].num_examples // BATCH_SIZE)
val_dataset = train_dataset.take(val_batches)
train_dataset = train_dataset.skip(val_batches)

# Load pre-trained MobileNetV2 model (without top classification layer)
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze base model

# Add custom classification layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train initial layers (only new head)
initial_epochs = 3
model.fit(train_dataset, validation_data=val_dataset, epochs=initial_epochs)

# Fine-tune: Unfreeze the base model
base_model.trainable = True

# Re-compile with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training with fine-tuning
fine_tune_epochs = 3
total_epochs = initial_epochs + fine_tune_epochs

model.fit(train_dataset, validation_data=val_dataset, epochs=total_epochs)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.8PNHNP_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 489s 817ms/step - accuracy: 0.9617 - loss: 0.0977 - val_accuracy: 0.9821 - val_loss: 0.0459
Epoch 2/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 475s 802ms/step - accuracy: 0.9825 - loss: 0.0462 - val_accuracy: 0.9843 - val_loss: 0.0456
Epoch 3/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 419s 706ms/step - accuracy: 0.9872 - loss: 0.0357 - val_accuracy: 0.9810 - val_loss: 0.0528
Epoch 1/6
582/582 ━━━━━━━━━━━━━━━━━━━━ 1802s 3s/step - accuracy: 0.9457 - loss: 0.1357 - val_accuracy: 0.9864 - val_loss: 0.0414
Epoch 2/6
 97/582 ━━━━━━━━━━━━━━━━━━━━ 23:21 3s/step - accuracy: 0.9679 - loss: 0.0769